In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

#year = 2022
name = 'MCS'
#buy_date = '2021-06-15' # oldest active record

In [2]:
format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','profit':'{:,.2f}',
              'pct':'{:,.2f}%','net':'{:,.2f}',
              'cost_amt':'{:,.2f}','unit_cost':'{:,.2f}','avg_cost':'{:,.2f}'}

pd.set_option('display.float_format','{:,.2f}'.format)

### Record selection for active stocks

In [3]:
sql = '''
SELECT name, buys.date AS buy_date,
buys.price AS unit_cost, qty, 
(buys.price * qty) AS cost_amt, net
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = '%s' AND status = 'Active'
ORDER BY buys.date DESC'''
sql = sql % name
print(sql)

buys_df = pd.read_sql(sql, conpf)
buys_df.style.format(format_dict)


SELECT name, buys.date AS buy_date,
buys.price AS unit_cost, qty, 
(buys.price * qty) AS cost_amt, net
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = 'MCS' AND status = 'Active'
ORDER BY buys.date DESC


,name,buy_date,unit_cost,qty,cost_amt,net
0,MCS,2022-04-19,12.80,"11,900","152,320.00","152,657.37"
1,MCS,2022-04-18,12.80,"13,100","167,680.00","168,051.40"
2,MCS,2017-04-01,16.70,"10,000","167,000.00","167,369.89"
3,MCS,2016-11-17,16.70,"10,000","167,000.00","167,369.89"
4,MCS,2016-11-14,16.70,"10,000","167,000.00","167,369.89"
5,MCS,2016-10-31,16.70,"10,000","167,000.00","167,369.89"
6,MCS,2016-10-13,16.70,"10,000","167,000.00","167,369.89"
7,MCS,2016-09-20,16.70,"10,000","167,000.00","167,369.89"


In [4]:
buys_df[['cost_amt','net','qty']].sum()

cost_amt   1,322,000.00
net        1,324,928.11
qty           85,000.00
dtype: float64

In [5]:
qty = 85_000
net_cost = 1_325_000
net_cost, qty

(1325000, 85000)

### Record selection for sold stocks

In [6]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = '%s' 
ORDER BY sell_year DESC, sell_month DESC, name'''

sql = sql % (name)
sells_df = pd.read_sql(sql, conpf)
print(sql)
sells_df


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = 'MCS' 
ORDER BY sell_year DESC, sell_month DESC, name


,name,sell_year,sell_month,buy_year,buy_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,profit,kind
0,MCS,2022,6,2022,6,11.70,11.40,0.30,5000,"58,500.00","57,000.00","1,500.00",2.63,"1,244.17",HD
1,MCS,2022,4,2022,4,12.50,12.10,0.40,15000,"187,500.00","181,500.00","6,000.00",3.31,"5,182.69",HD
2,MCS,2022,2,2021,3,14.70,16.70,-2.00,5000,"73,500.00","83,500.00","-10,000.00",-11.98,"-10,347.75",HD
3,MCS,2022,1,2019,11,14.60,16.70,-2.10,5000,"73,000.00","83,500.00","-10,500.00",-12.57,"-10,846.64",HD
4,MCS,2021,12,2016,8,14.70,16.70,-2.00,10000,"147,000.00","167,000.00","-20,000.00",-11.98,"-20,695.48",HD
5,MCS,2021,6,2016,8,14.90,16.70,-1.80,10000,"149,000.00","167,000.00","-18,000.00",-10.78,"-18,699.91",HD
6,MCS,2018,9,2016,12,7.20,17.40,-10.20,5000,"36,000.00","87,000.00","-51,000.00",-58.62,"-51,272.44",XP
7,MCS,2018,9,2016,12,7.30,17.40,-10.10,5000,"36,500.00","87,000.00","-50,500.00",-58.05,"-50,773.55",XP


In [7]:
sells_df[['gross','profit','qty']].sum()

gross    -152,500.00
profit   -156,208.91
qty        60,000.00
dtype: float64

In [8]:
profit = -156_200

### Record selection for dividends

In [9]:
sql = '''
SELECT name, p_date, amt, net, amt-net AS refund,
year, quarter AS q, x_date, number, ppu, (net / number * 1000) AS div_1k
FROM dividends
WHERE name = '%s' 
ORDER BY p_date DESC'''
sql = sql % (name)
print(sql)

div_df = pd.read_sql(sql, conpf)
div_df


SELECT name, p_date, amt, net, amt-net AS refund,
year, quarter AS q, x_date, number, ppu, (net / number * 1000) AS div_1k
FROM dividends
WHERE name = 'MCS' 
ORDER BY p_date DESC


,name,p_date,amt,net,refund,year,q,x_date,number,ppu,div_1k
0,MCS,2022-05-05,"25,000.00","24,500.00",500.00,2021,4,2022-03-15,50000,0.50,490.00
1,MCS,2021-12-09,"25,900.00","25,200.00",700.00,2021,3,2021-11-25,70000,0.37,360.00
2,MCS,2021-09-15,"28,000.00","26,460.00","1,540.00",2021,2,2021-09-03,70000,0.40,378.00
3,MCS,2021-04-28,"45,000.00","43,050.00","1,950.00",2020,4,2021-03-15,75000,0.60,574.00
4,MCS,2020-09-10,"30,000.00","29,325.00",675.00,2020,2,2020-08-27,75000,0.40,391.00
5,MCS,2020-04-29,"30,000.00","29,250.00",750.00,2019,4,2020-04-29,75000,0.40,390.00
6,MCS,2019-12-04,"18,750.00","18,525.00",225.00,2019,3,2019-11-21,75000,0.25,247.00
7,MCS,2019-04-29,"35,000.00","34,370.00",630.00,2018,4,2019-03-12,70000,0.50,491.00
8,MCS,2018-05-07,"32,000.00","29,840.00","2,160.00",2017,4,2018-04-18,80000,0.40,373.00
9,MCS,2017-09-11,"16,000.00","14,400.00","1,600.00",2017,2,2017-08-24,80000,0.20,180.00


In [10]:
div_grp = div_df.groupby(['name'], as_index=True).agg(
    {
        'amt':['sum'],
        'net':['sum'], 
        'number':['mean','count'],         
    }
)
div_grp

amt        net    number      
            sum        sum      mean count
name                                      
MCS  324,050.00 312,680.00 72,727.27    11

In [11]:
acc_div = 312_700
acc_div

312700

In [12]:
sell_amt = net_cost - acc_div - profit
sell_amt, qty

(1168500, 85000)

In [13]:
sell_price = sell_amt / qty
sell_price

13.74705882352941

In [14]:
sell_price = 13.75 # + unit_plus
sales_amt = sell_price * qty
sales_amt

1168750.0

### Plan A: Sell at 13.75 x 85,000 shares